In [2]:
from core.utils import Tibanna
from core import ff_utils
from invoke import run
import time
from datetime import datetime

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)

intr = "intermediate file"

Converter = {
    "bwa-mem 0.2.5":{"bam":intr},
    "hi-c-processing-bam 0.2.5":{"bam": "alignment",
                                 "pairs": 'contact list-replicate'},
    "hi-c-processing-pairs 0.2.5":{"normvector_juicerformat": "juicebox norm vector",
                                   "hic": "contact matrix",
                                   "mcool":"contact matrix",
                                   "pairs":"contact list-combined"}

#     "bwa-mem":{"bam":intr},
#     "pairsam-parse-sort":{"pairsam":intr},
#     "pairsam-merge":{"pairsam":intr},
#     "pairsam-markasdup":{"pairsam":intr},
#     "pairsam-filter":{"pairs":intr,"bam":"alignment"},
#     "addfragtopairs":{"pairs":intr},
#     "pairs-patch":{"pairs":"contact list"},
#     "hi-c-processing-partb exp": {"hic": "contact matrix","cool":intr, "pairs":intr},
#     "hi-c-processing-partc exp": {"mcool":"contact matrix", "normvector_juicerformat":"juicebox norm vector"},
#     "hi-c-processing-partb set": {"hic": "contact matrix","cool":intr, "pairs":intr},
#     "hi-c-processing-partc set": {"mcool":"contact matrix", "normvector_juicerformat":"juicebox norm vector"}
}

#tibanna.ff_keys['default']['server'] = 'http://staging.4dnucleome.org'
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'


ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
my_url = '/search/?sort=-date_created&type=WorkflowRunAwsem&workflow.title=Alignment%20for%20Hi-C%20using%20bwa-mem&workflow.title=Hi-C%20Post-alignment%20Processing&workflow.title=Generation%20of%20multiresolution%20Hi-C%20contact%20matrices%20from%20a%20set%20of%20contact%20lists'
all_aws = ff_utils.get_metadata(my_url, connection=ff)['@graph']
print len(all_aws)

1047


In [3]:
my_wfrs = Converter.keys()
print my_wfrs

out_n = "This is an output file of the Hi-C processing pipeline"
int_n = "This is an intermediate file in the HiC processing pipeline"

success = 0
patch_problem = 0
deleted = 0
for wfr_resp in all_aws:
    if wfr_resp['status'] == 'deleted':
        deleted +=1
        continue
        
    wfr_name = wfr_resp['display_title'].split(' run')[0]
    if wfr_name in my_wfrs:
        outputs = wfr_resp.get('output_files')
        for output in outputs:
            
            form = output['format']
            file_id = output['value']
            f_type =  Converter[wfr_name][form]
            
            desc = ""
            if f_type == "intermediate file":
                desc = int_n
            else:
                desc = out_n
                
            patch_data = {"description": desc, 'file_type': f_type}
            try:
                #print file_id
                #print(ff_utils.get_metadata(file_id, connection=ff)['uuid'])
                ff_utils.patch_metadata(patch_data, obj_id=file_id ,connection=ff)
                success += 1
                print success
            except Exception as e:
                print e
                print
                patch_problem += 1

            
print success
print patch_problem
print deleted


['hi-c-processing-bam 0.2.5', 'bwa-mem 0.2.5', 'hi-c-processing-pairs 0.2.5']
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143


KeyboardInterrupt: 